
 
<img src="images/banner2.png" alt="banner" width="150" height="100">

# **Verzamelen:** weerdata Noordoostpolder 
Voor het eindproject wil ik weerdata gebruiken om specifiek voor de dorpen in de Noordoostpolder een aantal vragen te beantwoorden.<br>
Deze worden beantwoord door gebruik te maken van 6 visuals in de analyse notebook. 

<img src="images/dorpen.jpg" alt="banner" width="523" height="483">

**3 historische vragen:**<br>
*Hoe ontwikkelde de gemiddelde dagtempratuur zich in de dorpen van de Noordoostpolder in de afgelopen 10 jaar?*<br>
*In welke dorp viel de meeste neerslag in 2025?*<br>
*Welke dorpen hadden de hoogste windsnelheden in de periode van 2020-2025?*

**3 actuele  vragen:**<br>
*In welk dorp is het nu het warmst/koudst?*<br>
*Wat is de verwachting m.b.t. tempratuur/neerslag/windsnelheid voor de komende dagen in de dorpen van de Noordoostpolder?*<br>
*Hoe groot zijn de verschillen in temperatuur, neerslag en windsnelheid tussen de dorpen van de Noordoostpolder vandaag?*

Hiervoor maak ik gebruik van 2 API`s: 
-	OpenMeteo API (voor historische data)
-	Met Norway API (voor actuele data)<br>

In dit notebook ga ik de data verzamelen door verbinding te maken met de APIs.<br>
Vervolgens gaan we de data transformeren (cleanen waar nodig) en wegschrijven naar CSV-bestanden, in de volgende notebook gaan we de databestanden weer inlezen<br>
en gebruiken om met matplotlib er visualisaties van te maken waarmee de bovenstaande 6 vragen worden beantwoord.

## Benodigde modules importeren

Voor dit notebook zijn de volgende modules nodig:<br>
* **requests** (voor het verbinden met de API en ophalen van de data)<br>
* **pandas** (voor het opschonen/aanpassen van de data en exporteren naar CSV)<br>
* **time** (voor tussenpauze te creeren tussen het ophalen van data voor de dorpen)<br>

In [1]:
# module imports
import requests
import pandas as pd
import time

<img src="images/openmeteo.jpg" alt="banner" width="125" height="125">

## **Open-Meteo API** - ETL proces
We beginnen met het ophalen van de data uit de Open-Meteo API, deze transformeren we naar bruikbare data en slaan dit op als CSV.<br>
Met de dataframe die we gaan maken kunnen we in de volgende notebook de historische vrgaen beantwoorden.<br>
Voor deze API is **geen** authentificatie nodig.


### Stap 1 - Extract

In [2]:
### ETL-proces --> Open-Meteo API ###

#####################################
### EXTRACT #########################
#####################################

# lijst met dorpen + coördinaten (latitude, longitude)
villages = {
    "Emmeloord": (52.710, 5.750),
    "Marknesse": (52.700, 5.850),
    "Ens": (52.650, 5.750),
    "Bant": (52.760, 5.750),
    "Creil": (52.780, 5.670),
    "Luttelgeest": (52.750, 5.900),
    "Rutten": (52.830, 5.700),
    "Tollebeek": (52.670, 5.670),
    "Nagele": (52.650, 5.650),
}

# definieren van de api url + response ophalen
url = BASE_URL = "https://archive-api.open-meteo.com/v1/archive"

# periode definiëren
START_DATE = "2015-01-01"
END_DATE = pd.Timestamp.now().strftime("%Y-%m-%d") # huidige datum gebruiken

# alle standaard variabelen die Open-Meteo ondersteunt opgeven
DAILY_VARIABLES = [
    "temperature_2m_max",
    "temperature_2m_min",
    "temperature_2m_mean",
    "precipitation_sum",
    "windspeed_10m_max",
    "windspeed_10m_mean",
    "sunshine_duration"
]

# functie om ruwe data op te halen
def fetch_openmeteo(village, lat, lon):
    params = {
        "latitude": lat, # voor juiste dorp
        "longitude": lon, # voor juiste dorp
        "start_date": START_DATE,
        "end_date": END_DATE,
        "daily": ",".join(DAILY_VARIABLES), # lijst omzetten naar string met komma's
        "timezone": "Europe/Amsterdam"
    }
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    data = response.json()
    
    # zet alles wat onder 'daily' zit naar DataFrame
    df = pd.DataFrame(data["daily"]) # JSON-sleutel waar alle data in staat
    df["dorp"] = village
    return df

# alle dorpen ophalen met for loop
all_data = []
for village, (lat, lon) in villages.items(): # door alle dorpen loopen
    print(f"Fetching data voor {village}...")
    df = fetch_openmeteo(village, lat, lon) # fetch def functie gebruiken om dataframe te vullen
    all_data.append(df) # sla dataframe op in lijst
    time.sleep(20) # 15 seconden wachten om rate limiting te voorkomen
 
# combineer alle dataframes in één grote dataframe
df1 = pd.concat(all_data, ignore_index=True)

# metadata toevoegen, bron + etl_timestamp
df1["source"] = "Open-Meteo"
df1["etl_timestamp"] = pd.Timestamp.now().isoformat()

print(df1.head())

Fetching data voor Emmeloord...
Fetching data voor Marknesse...
Fetching data voor Ens...
Fetching data voor Bant...
Fetching data voor Creil...
Fetching data voor Luttelgeest...
Fetching data voor Rutten...
Fetching data voor Tollebeek...
Fetching data voor Nagele...
         time  temperature_2m_max  temperature_2m_min  temperature_2m_mean  \
0  2015-01-01                 5.1                 1.8                  3.3   
1  2015-01-02                 9.7                 4.9                  7.0   
2  2015-01-03                 5.8                 4.0                  5.1   
3  2015-01-04                 6.6                 2.7                  4.9   
4  2015-01-05                 4.1                 2.3                  3.4   

   precipitation_sum  windspeed_10m_max  windspeed_10m_mean  \
0                0.5               40.6                28.8   
1                5.4               45.3                38.6   
2                1.5               33.5                25.1   
3         

### Stap 2 - Transfer

In [3]:
#####################################
### TRANSFER ########################
#####################################

# dataframe aanroepen
#df1.head(50)

# kolommen selecteren
df1_filter = df1[[
    "time",
    "temperature_2m_max",
    "temperature_2m_min",
    "temperature_2m_mean",
    "precipitation_sum",
    "windspeed_10m_max",
    "windspeed_10m_mean",
    "dorp",
    "source",
    "etl_timestamp"
]]

# kolommen hernoemen
df1_rename = df1_filter.rename(columns={
        "time": "datum",
        "temperature_2m_max": "max_temp",
        "temperature_2m_min": "min_temp",
        "temperature_2m_mean": "gem_temp",
        "precipitation_sum": "neerslag_mm",
        "windspeed_10m_max": "max_windsnelheid",
        "windspeed_10m_mean": "gem_windsnelheid",
        "source": "data_bron",
            })

# data types aanpassen
df1_rename["datum"] = pd.to_datetime(df1_rename["datum"])
df1_rename["max_temp"] = df1_rename["max_temp"].astype(float)
df1_rename["min_temp"] = df1_rename["min_temp"].astype(float)
df1_rename["gem_temp"] = df1_rename["gem_temp"].astype(float)
df1_rename["neerslag_mm"] = df1_rename["neerslag_mm"].astype(float)
df1_rename["max_windsnelheid"] = df1_rename["max_windsnelheid"].astype(float)
df1_rename["gem_windsnelheid"] = df1_rename["gem_windsnelheid"].astype(float)
df1_rename["dorp"] = df1_rename["dorp"].astype(str)
df1_rename["data_bron"] = df1_rename["data_bron"].astype(str)

# nieuwe kolommen aanmaken: jaar, maand, jaar_maand
df1_rename["jaar"] = df1_rename["datum"].dt.year
df1_rename["maand"] = df1_rename["datum"].dt.month
df1_rename["jaar_maand"] = df1_rename["datum"].dt.to_period("M")

# kolommen opnieuw schikken
df1_filter2 = df1_rename[[
    "datum",
    "jaar_maand",
    "maand",
    "jaar",
    "max_temp",
    "min_temp",
    "gem_temp",
    "neerslag_mm",
    "max_windsnelheid",
    "gem_windsnelheid",
    "dorp",
    "data_bron",
    "etl_timestamp"  
]]

# uiteindelijke schone dataframe
df1_cleaned = df1_rename.copy() 

# dataframe info, beschrijving en eerste 10 rijen
df1_cleaned.info()
df1_cleaned.describe()
df1_cleaned.head(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36099 entries, 0 to 36098
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   datum             36099 non-null  datetime64[ns]
 1   max_temp          36099 non-null  float64       
 2   min_temp          36099 non-null  float64       
 3   gem_temp          36099 non-null  float64       
 4   neerslag_mm       36099 non-null  float64       
 5   max_windsnelheid  36099 non-null  float64       
 6   gem_windsnelheid  36099 non-null  float64       
 7   dorp              36099 non-null  object        
 8   data_bron         36099 non-null  object        
 9   etl_timestamp     36099 non-null  object        
 10  jaar              36099 non-null  int32         
 11  maand             36099 non-null  int32         
 12  jaar_maand        36099 non-null  period[M]     
dtypes: datetime64[ns](1), float64(6), int32(2), object(3), period[M](1)
memory u

,datum,max_temp,min_temp,gem_temp,neerslag_mm,max_windsnelheid,gem_windsnelheid,dorp,data_bron,etl_timestamp,jaar,maand,jaar_maand
0,2015-01-01,5.1,1.8,3.3,0.5,40.6,28.8,Emmeloord,Open-Meteo,2025-12-24T13:24:55.470493,2015,1,2015-01
1,2015-01-02,9.7,4.9,7.0,5.4,45.3,38.6,Emmeloord,Open-Meteo,2025-12-24T13:24:55.470493,2015,1,2015-01
2,2015-01-03,5.8,4.0,5.1,1.5,33.5,25.1,Emmeloord,Open-Meteo,2025-12-24T13:24:55.470493,2015,1,2015-01
3,2015-01-04,6.6,2.7,4.9,0.0,27.1,20.7,Emmeloord,Open-Meteo,2025-12-24T13:24:55.470493,2015,1,2015-01
4,2015-01-05,4.1,2.3,3.4,0.0,19.8,17.6,Emmeloord,Open-Meteo,2025-12-24T13:24:55.470493,2015,1,2015-01


### Stap 3 - Load

In [4]:
#####################################
### LOAD ############################
#####################################

# bestansnaam
filename = f"data/noordoostpolder_openmeteo_clean_2015_2025_latest_version.csv"

# wegschrijven naar CSV
df1_cleaned.to_csv(filename, index=False)

# bevestiging van succesvolle uitvoering ETL-proces
print("Data Open-Meteo succesvol opgehaald, getransformeerd en opgeslagen!")


Data Open-Meteo succesvol opgehaald, getransformeerd en opgeslagen!


<img src="images/metnorway.png" alt="banner" width="230" height="114">

## **MET Norway API** - ETL proces
Nu gaan we verder met de MET Norway API en doen in principe dezelfde acties: ophalen, transformeren, wegschrijven (ETL)<br>
Met deze gegevens gaan we de actuele vragen beantwoorden <br>

Voor deze API verbinding is **geen** een API-key nodig.

### Stap 1 - Extract

In [5]:
### ETL-proces --> Met Norway API ###

######################################
### EXTRACT ##########################
######################################


# lijst met dorpen + coördinaten (latitude, longitude)
villages = {
    "Emmeloord": (52.710, 5.748),
    "Marknesse": (52.707, 5.833),
    "Creil": (52.780, 5.660),
    "Nagele": (52.650, 5.700),
    "Tollebeek": (52.670, 5.720),
    "Luttelgeest": (52.760, 5.830),
    "Ens": (52.650, 5.830),
    "Bant": (52.780, 5.720),
    "Rutten": (52.830, 5.700)
}

# custom user-agent volgens API richtlijnen
headers = {
    "User-Agent": "Tim-Weather-ETL/1.0 (contact: example@example.com)"
}

# basis URL voor locatieforecast API
base_url = "https://api.met.no/weatherapi/locationforecast/2.0/compact"


# lege lijst voor uurlijkse records per dag en dorp
hourly_records = []

for village, (lat, lon) in villages.items():
    print(f"Fetching data voor {village}...")
    url = f"{base_url}?lat={lat}&lon={lon}"
    data = requests.get(url, headers=headers).json()

    timeseries = data["properties"]["timeseries"]

    for entry in timeseries:
        t = entry["time"]
        instant = entry["data"]["instant"]["details"]
        next1h = entry["data"].get("next_1_hours", {}).get("details", {})

        hourly_records.append({
            "village": village,
            "time": t,
            "temperature": instant.get("air_temperature"),
            "windspeed": instant.get("wind_speed"),
            "precipitation": next1h.get("precipitation_amount")
        })


# lijst omzetten naar dataframe
df2 = pd.DataFrame(hourly_records)

# metadata toevoegen, bron + etl_timestamp
df2["source"] = "Met Norway API"
df2["etl_timestamp"] = pd.Timestamp.now().isoformat()

print(df2.head())

Fetching data voor Emmeloord...
Fetching data voor Marknesse...
Fetching data voor Creil...
Fetching data voor Nagele...
Fetching data voor Tollebeek...
Fetching data voor Luttelgeest...
Fetching data voor Ens...
Fetching data voor Bant...
Fetching data voor Rutten...
     village                  time  temperature  windspeed  precipitation  \
0  Emmeloord  2025-12-24T12:00:00Z          2.5        6.6            0.0   
1  Emmeloord  2025-12-24T13:00:00Z          2.7        6.7            0.0   
2  Emmeloord  2025-12-24T14:00:00Z          2.5        7.1            0.0   
3  Emmeloord  2025-12-24T15:00:00Z          1.8        6.6            0.0   
4  Emmeloord  2025-12-24T16:00:00Z          0.9        5.9            0.0   

           source               etl_timestamp  
0  Met Norway API  2025-12-24T13:24:57.984793  
1  Met Norway API  2025-12-24T13:24:57.984793  
2  Met Norway API  2025-12-24T13:24:57.984793  
3  Met Norway API  2025-12-24T13:24:57.984793  
4  Met Norway API  2025-12-2

### Stap 2 - Transfer

In [6]:
#####################################
### TRANSFER ########################
#####################################

# dataframe aanroepen
df2.head(50)

# kolommen selecteren
df2_filter = df2[[
    "time",
    "temperature",
    "precipitation",
    "windspeed",
    "village",
    "source",
    "etl_timestamp"  
]]

# kolommen hernoemen
df2_rename = df2_filter.rename(columns={
        "time": "datum",
        "temperature": "temp",
        "precipitation": "neerslag_mm",
        "windspeed": "windsnelheid",
        "village": "dorp",
        "source": "data_bron"
            })

# data types aanpassen
df2_rename["datum"] = pd.to_datetime(df2_rename["datum"])
df2_rename["temp"] = df2_rename["temp"].astype(float)
df2_rename["neerslag_mm"] = df2_rename["neerslag_mm"].astype(float)
df2_rename["windsnelheid"] = df2_rename["windsnelheid"].astype(float)
df2_rename["dorp"] = df2_rename["dorp"].astype(str)
df2_rename["data_bron"] = df2_rename["data_bron"].astype(str)

# extra kolommen voor dag en uur
df2_rename["dag"] = df2_rename["datum"].dt.date
df2_rename["uur"] = df2_rename["datum"].dt.hour
df2_rename["uur"] = df2_rename["datum"].dt.strftime("%H:%M")

# kolommen opnieuw schikken
df2_filter2 = df2_rename[[
    "datum",
    "dag",
    "uur",
    "temp",
    "neerslag_mm",
    "windsnelheid",
    "dorp",
    "data_bron",
    "etl_timestamp"  
]]

# windsnelheid naar km/u eenheid zetten
df2_filter2["windsnelheid"] = df2_filter2["windsnelheid"] * 3.6

# uiteindelijke schone dataframe
df2_cleaned = df2_filter2.copy() 

# dataframe info, beschrijving en eerste 10 rijen
df2_cleaned.info()
df2_cleaned.describe()
df2_cleaned.head(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773 entries, 0 to 772
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   datum          773 non-null    datetime64[ns, UTC]
 1   dag            773 non-null    object             
 2   uur            773 non-null    object             
 3   temp           773 non-null    float64            
 4   neerslag_mm    498 non-null    float64            
 5   windsnelheid   773 non-null    float64            
 6   dorp           773 non-null    object             
 7   data_bron      773 non-null    object             
 8   etl_timestamp  773 non-null    object             
dtypes: datetime64[ns, UTC](1), float64(3), object(5)
memory usage: 54.5+ KB


,datum,dag,uur,temp,neerslag_mm,windsnelheid,dorp,data_bron,etl_timestamp
0,2025-12-24 12:00:00+00:00,2025-12-24,12:00,2.5,0.0,23.76,Emmeloord,Met Norway API,2025-12-24T13:24:57.984793
1,2025-12-24 13:00:00+00:00,2025-12-24,13:00,2.7,0.0,24.12,Emmeloord,Met Norway API,2025-12-24T13:24:57.984793
2,2025-12-24 14:00:00+00:00,2025-12-24,14:00,2.5,0.0,25.56,Emmeloord,Met Norway API,2025-12-24T13:24:57.984793
3,2025-12-24 15:00:00+00:00,2025-12-24,15:00,1.8,0.0,23.76,Emmeloord,Met Norway API,2025-12-24T13:24:57.984793
4,2025-12-24 16:00:00+00:00,2025-12-24,16:00,0.9,0.0,21.24,Emmeloord,Met Norway API,2025-12-24T13:24:57.984793


### Stap 3 - Load

In [7]:
#####################################
### LOAD ############################
#####################################

# bestansnaam
filename = f"data/noordoostpolder_metnorway_clean_2015_2025_latest_version.csv"

# wegschrijven naar CSV
df2_cleaned.to_csv(filename, index=False)

# bevestiging van succesvolle uitvoering ETL-proces
print("Data Met Norway API succesvol opgehaald, getransformeerd en opgeslagen!")

Data Met Norway API succesvol opgehaald, getransformeerd en opgeslagen!
